In [33]:
import pandas as pd
from data.raw.club_mapping import club_mapping
from modules.calculate_average_elo import calculate_average_elo

club_elo = pd.read_csv("data/processed/club_elo.csv")

club_elo.set_index("club_id")["elo_rating"][1]

min_matchday = 30
max_matchday = 30

club_average_opponent_elo = calculate_average_elo(min_matchday, max_matchday)


In [34]:
club_average_opponent_elo


,club_id,club_name,club_elo,average_home_opponent_elo,average_away_opponent_elo,average_opponent_elo
15,16,1. FC Slovácko,1359,NaN,1695.0,1695.0
3,4,FK Pardubice,1284,1634.0,NaN,1634.0
14,15,FK Dukla Praha,1238,NaN,1614.0,1614.0
2,3,Bohemians 1905,1365,NaN,1501.0,1501.0
1,2,FK Mladá Boleslav,1430,1433.0,NaN,1433.0
6,7,FK Jablonec,1433,NaN,1430.0,1430.0
10,11,FK Teplice,1348,1410.0,NaN,1410.0
9,10,SK Dynamo České Budějovice,1186,NaN,1401.0,1401.0
8,9,MFK Karviná,1312,NaN,1395.0,1395.0
4,5,FC Baník Ostrava,1501,1365.0,NaN,1365.0


In [4]:
import pandas as pd
from data.raw.club_mapping import club_mapping

club_mapping



fixtures_clubs = {}
club_average_opponent_elo = pd.DataFrame(columns=["club_name", "club_elo", "average_home_opponent_elo", "average_away_opponent_elo", "average_opponent_elo"])



for club in czech_clubs["club_name"]:
    fixtures_clubs[club] = fixtures[(fixtures["home_team"] == club) | (fixtures["away_team"] == club)]
    fixtures_clubs[club] = fixtures_clubs[club].sort_values(by="event_timestamp")
    fixtures_clubs[club]["home_away"] = fixtures_clubs[club].apply(lambda x: "home" if x["home_team"] == club else "away", axis=1)
    fixtures_clubs[club]["opponent_elo"] = fixtures_clubs[club].apply(lambda x: df_club_elo[df_club_elo["club_name"] == x["away_team"]]["elo_rating"].values[0] if x["home_away"] == "home" else df_club_elo[df_club_elo["club_name"] == x["home_team"]]["elo_rating"].values[0], axis=1)
    fixtures_clubs[club]["club_elo"] = df_club_elo[df_club_elo["club_name"] == club]["elo_rating"].values[0]

    future_fixtures = fixtures_clubs[club][fixtures_clubs[club]["is_planned_tf"]]

    club_name = club
    club_elo = fixtures_clubs[club]["club_elo"].max().astype(int)
    average_home_opponent_elo = future_fixtures[future_fixtures['home_away'] == 'home']['opponent_elo'].mean()
    average_away_opponent_elo = future_fixtures[future_fixtures['home_away'] == 'away']['opponent_elo'].mean()
    average_opponent_elo = future_fixtures['opponent_elo'].mean()


    club_average_opponent_elo.loc[len(club_average_opponent_elo)] = [
        club_name,
        club_elo,
        average_home_opponent_elo,
        average_away_opponent_elo,
        average_opponent_elo
    ]

club_average_opponent_elo.sort_values(by="average_opponent_elo", ascending=False, inplace=True)
club_average_opponent_elo.to_csv("data/processed/club_average_opponent_elo.csv", index=False)

